# Vehicle Classification

## Evaluate your model


In [1]:
import os

from sklearn.metrics import accuracy_score, classification_report

from utils import utils
from models import resnet_50


## Experiment settings

Set here the two variables in the following way:

- **CONFIG_YML:** assign the path to the config.yml file used for the experiment you want to evaluate
- **WEIGHTS:** assign the path to the model weights (.h5 file) you want to evaluate


In [2]:
# Modify this
CONFIG_YML = "../experiments/exp_001/config.yml"
WEIGHTS = "../experiments/exp_001/model.10-1.4167.h5"

In [3]:
# (!) Don't touch this cell, will load the config file specified before
config = utils.load_config(CONFIG_YML)
config

{'seed': 123,
 'data': {'directory': '/home/app/src/data/car_ims_v2/train',
  'labels': 'inferred',
  'label_mode': 'categorical',
  'validation_split': 0.2,
  'image_size': [224, 224],
  'batch_size': 32},
 'model': {'weights': 'imagenet',
  'input_shape': [224, 224, 3],
  'classes': 196,
  'droput_rate': 0.5,
  'data_aug_layer': {'random_flip': {'mode': 'horizontal'},
   'random_rotation': {'factor': 0.5},
   'random_zoom': {'height_factor': 0.5, 'width_factor': 0.5}}},
 'compile': {'optimizer': {'adam': {'learning_rate': 0.0001}},
  'loss': 'categorical_crossentropy',
  'metrics': ['accuracy']},
 'fit': {'epochs': 50,
  'callbacks': {'model_checkpoint': {'filepath': '/home/app/src/experiments/exp_002/model.{epoch:02d}-{val_loss:.4f}.h5',
    'save_best_only': True},
   'tensor_board': {'log_dir': '/home/app/src/experiments/exp_002/logs'}}}}

In [4]:
# (!) Don't touch this cell, will use the config file to infer the class names
#     and also to load the corresponding testing dataset.
#     If you get an error, you must review your data/code.

MODEL_CLASSES = utils.get_class_names(config)

if len(MODEL_CLASSES) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your data!"
    )

_dirname, _ = os.path.split(config['data']['directory'])
TEST_FOLDER = os.path.join(_dirname, 'test')

if not os.path.exists(TEST_FOLDER):
    raise ValueError("'test' folder not found!")
    
if len(os.listdir(TEST_FOLDER)) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your testing dataset!"
    )
    
if set(os.listdir(TEST_FOLDER)) != set(MODEL_CLASSES):
    raise ValueError(
        "The name of the subfolders inside your test set "
        "doesn't match with the model classes!"
    )

## Load your model

Use `resnet_50.create_model()` and remember to properly setup the model weights!

Assign the model to the variable `cnn_model`.


In [5]:
# Complete this
# TODO
cnn_model = None

# It should print your model correctly
print(cnn_model.summary())

2022-04-02 00:09:51.860920: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-02 00:09:51.869532: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-02 00:09:51.870049: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-02 00:09:51.871497: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                             

## Get predictions from testing dataset

In [7]:
# (!) Don't touch this cell, will use the loaded model and
#     the function utils.predict_from_folder() to get 
#     model predictions and the corresponding true labels
#     so we can measure the accuracy

predictions, labels = utils.predict_from_folder(
    folder=TEST_FOLDER, 
    model=cnn_model, 
    input_size=config["data"]["image_size"], 
    class_names=MODEL_CLASSES,
)

if len(predictions) != len(labels):
    raise ValueError(
        "The lenght of predictions and labels lists doesn't match!"
    )

if not isinstance(predictions[0], str):
    raise ValueError(
        "Model predictions should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )

if not isinstance(labels[0], str):
    raise ValueError(
        "Ground true labels should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )


  0%|                                                                                                                                                                                       | 0/252 [00:00<?, ?it/s]2022-04-02 00:09:59.846378: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200
2022-04-02 00:10:00.358550: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-04-02 00:10:00.359121: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-04-02 00:10:00.359166: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-04-02 00:10:00.359832: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-04-02 00:10:00.359939: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Fa

In [8]:
# (!) Don't touch this cell

print(classification_report(y_true=labels, y_pred=predictions))


                                                        precision    recall  f1-score   support

                            AM General Hummer SUV 2000       0.57      0.70      0.63        44
                             Acura Integra Type R 2001       0.86      0.41      0.55        44
                                   Acura RL Sedan 2012       0.67      0.31      0.43        32
                                   Acura TL Sedan 2012       0.79      0.35      0.48        43
                                  Acura TL Type-S 2008       0.71      0.52      0.60        42
                                  Acura TSX Sedan 2012       0.77      0.25      0.38        40
                              Acura ZDX Hatchback 2012       0.31      0.62      0.41        39
              Aston Martin V8 Vantage Convertible 2012       0.32      0.56      0.41        45
                    Aston Martin V8 Vantage Coupe 2012       0.83      0.12      0.21        41
                  Aston Martin Virage C

In [9]:
# (!) Don't touch this cell

acc = accuracy_score(y_true=labels, y_pred=predictions)

print(f"Your model accuracy is {acc:.4f}!")

if acc < .3:
    raise ValueError("Your model accuracy is too low :(\nYou can do it better! :)")


Your model accuracy is 0.5394!
